In [4]:
%%capture
! pip install 'git+https://github.com/PyTorchLightning/lightning-flash.git#egg=lightning-flash[image]'

In [54]:
import flash
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier
from glob import glob
from pathlib import Path
import random

In [55]:
total_images = glob("../input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/**/*.jpg")
total_images = [Path(img) for img in total_images]
random.shuffle(total_images)
total_images[:5]

[PosixPath('../input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/auditorium/sisli_1__174.jpg'),
 PosixPath('../input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/bedroom/IMG_1098.jpg'),
 PosixPath('../input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/laundromat/lavanderia_74_21_flickr.jpg'),
 PosixPath('../input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/deli/deli_165_16_flickr.jpg'),
 PosixPath('../input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/livingroom/3Raised_salon_stbside.jpg')]

In [56]:
len(total_images), int(len(total_images)*0.60)

(15613, 9367)

**Splitting images into train, valid, and test**

In [76]:
train_img = total_images[:int(len(total_images)*0.80)]
valid_img = total_images[int(len(total_images)*0.80):]

len(train_img), len(valid_img)

(12490, 3123)

In [77]:
train_lbl = [img.parent.name for img in train_img]
valid_lbl = [img.parent.name for img in valid_img]

In [86]:
datamodule = ImageClassificationData.from_files(train_files=train_img, train_targets=train_lbl, 
                                                val_files=valid_img, val_targets=valid_lbl, 
                                                transform_kwargs=dict(image_size=(256, 256)),
                                                batch_size=32)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:74: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7."


In [87]:
datamodule.num_classes

67

In [88]:
model = ImageClassifier(backbone="resnet34", num_classes=datamodule.num_classes)

In [89]:
from flash import Trainer
trainer = Trainer(fast_dev_run=True, max_epochs=5)

In [84]:
trainer.fit(model, datamodule=datamodule)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  f"DataModule.{name} has already been called, so it will not be called again. "


In [47]:
datamodule = ImageClassificationData.from_files(train_files=train_img, train_targets=train_lbl, 
                                                val_files=valid_img, val_targets=valid_lbl,
                                                transform_kwargs=dict(image_size=(256, 256)),
                                                batch_size=32)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:74: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7."


In [46]:
trainer.fit(model, datamodule=datamodule)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")


In [90]:
trainer.finetune(model, datamodule=datamodule)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]